In [4]:
import os
import re
from itertools import chain

import pandas as pd
import numpy as np
import json
import logger
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import (
    AdamW,
    MT5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer,
    get_linear_schedule_with_warmup
    
)

In [12]:
def load_data(path, sep = '\t'):
    
    data = []
    with open(path, 'r', encoding = 'utf-8') as f: 
        for datum in tqdm(f):
            data.append(json.loads(datum))
            
    dataframe = pd.DataFrame(data)
    dataframe.dropna(inplace = True)
    
    return dataframe

In [5]:
class NewsSummaryDataset(Dataset):
    
    def __init__(
            self, 
            data: pd.DataFrame, 
            tokenizer: T5Tokenizer, 
            text_max_token_length: int = 512, 
            summary_max_token_length: int = 192
            ):
        
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_length = text_max_token_length
        self.summary_max_token_length = summary_max_token_length
        
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index: int):
        
        data_row = self.data.iloc[index]
        
        encoded_article = tokenizer(
            ' '.join(data_row.article_original),
            max_length = self.text_max_token_length, 
            padding = 'max_length', 
            truncation = True, 
            return_attention_mask = True, 
            add_special_tokens = True, 
            return_tensors = 'pt'
            )
        
        
        encoded_summarized_article = tokenizer(
            data_row.abstractive,
            max_length = self.text_max_token_length,
            padding = 'max_length',
            truncation = True,
            return_attention_mask = True,
            add_special_tokens = True,
            return_tensors = 'pt'
            )
        
        
        labels = encoded_summarized_article.input_ids
        labels[labels == 0] = -100
        
        
        return dict(
            original_article = ' '.join(data_row['article_original']),
            summary = data_row['abstractive'],
            text_input_ids = encoded_article['input_ids'].flatten(),
            text_attention_mask = encoded_article['attention_mask'].flatten(),
            labels = labels.flatten(),
            labels_attention_mask = encoded_summarized_article['attention_mask'].flatten(),
            )

In [6]:
class NewsSummaryDataModule(pl.LightningDataModule):
    
    def __init__(            
        self,
        train_df: pd.DataFrame,
        test_df: pd.DataFrame,
        tokenizer: T5Tokenizer,
        batch_size: int = 8,
        text_max_token_length: int = 512,
        summary_max_token_length: int = 192
    ):
    
        super().__init__()
        
        self.train_df = train_df
        self.test_df = test_df
        
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.text_max_token_length = text_max_token_length,
        self.summary_max_token_length = summary_max_token_length
        
        self.setup()
    
        
    def train_dataloader(self):        
        return DataLoader(
            self.train_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )
    
    
    def val_dataloader(self):        
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )
    
    
    def test_dataloader(self):        
        return DataLoader(
            self.test_dataset,
            batch_size = self.batch_size,
            shuffle = False
            )

In [7]:
class NewsSummaryModel(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small', return_dict = True)
        
        
    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels = None):
        
        output = self.model(
            input_ids,
            attention_mask = attention_mask,
            labels = labels,
            decoder_attention_mask = decoder_attention_mask
            )
       
        return output.loss, output.logits
    
    
    def training_step(self, batch, batch_index):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']
        
        loss, outputs = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_attention_mask = labels_attention_mask,
            labels = labels)
        
        self.log('train_loss', loss, prog_bar = True, logger = True)
        
        return loss
    
    
    def validation_step(self, batch, batch_index):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']
        
        loss, outputs = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_attention_mask = labels_attention_mask,
            labels = labels)
        
        self.log('val_loss', loss, prog_bar = True, logger = True)
        
        return loss
    
    
    def test_step(self, batch, batch_index):
        input_ids = batch['text_input_ids']
        attention_mask = batch['text_attention_mask']
        labels = batch['labels']
        labels_attention_mask = batch['labels_attention_mask']
        
        loss, outputs = self(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_attention_mask = labels_attention_mask,
            labels = labels)
        
        self.log('test_loss', loss, prog_bar = True, logger = True)
        
        return loss
    
    
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr = 1e-5)

In [13]:
data = load_data('문서요약 텍스트/1.Training/신문기사/train.jsonl')
train, test = train_test_split(data, test_size = 0.1)

240972it [00:08, 27446.61it/s]


In [14]:
tokenizer = T5Tokenizer.from_pretrained('google/mt5-small')

EPOCHS = 10
BATCH_SIZE = 8

In [20]:
data_module = NewsSummaryDataModule(train, test, tokenizer, batch_size = BATCH_SIZE)

model = NewsSummaryModel()

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)


In [21]:
checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'best-checkpoint',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min'
    )

logger = TensorBoardLogger('lightning_logs', name = 'news-summary')

trainer = pl.Trainer(
    logger = logger,
    checkpoint_callback = checkpoint_callback,
    max_epochs = EPOCHS,    
    progress_bar_refresh_rate = 1,
    gpus = 2,
    accelerator = 'dp',
    )

[20/Jul/2021 10:55:47] INFO - GPU available: True, used: True
[20/Jul/2021 10:55:47] INFO - TPU available: False, using: 0 TPU cores


In [42]:
trainer.fit(model, data_module)

[24/Jun/2021 10:35:20] INFO - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
[24/Jun/2021 10:35:25] INFO - 
  | Name  | Type                        | Params
------------------------------------------------------
0 | model | MT5ForConditionalGeneration | 300 M 
------------------------------------------------------
300 M     Trainable params
0         Non-trainable params
300 M     Total params
1,200.707 Total estimated model params size (MB)


/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

[26/Jun/2021 04:37:09] INFO - Epoch 8, global step 243989: val_loss reached 1.17147 (best 1.17147), saving model to "/home/ubuntu/Documents/python_code/summarization/t5_summarization/checkpoints/best-checkpoint.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

[26/Jun/2021 09:22:38] INFO - Epoch 9, global step 271099: val_loss reached 1.16021 (best 1.16021), saving model to "/home/ubuntu/Documents/python_code/summarization/t5_summarization/checkpoints/best-checkpoint.ckpt" as top 1


1

In [26]:
summarize_model = NewsSummaryModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
summarize_model.freeze()

In [28]:
def summarize(text):    
    text_encoding = tokenizer.encode_plus(
        test_comment,     
        max_length = 512,
        padding = 'max_length',
        truncation = True,
        return_attention_mask = True,
        add_special_tokens = True,
        return_tensors = 'pt'
    )
    
    generated_ids = summarize_model.model.generate(
        input_ids = text_encoding.input_ids,
        attention_mask = text_encoding.attention_mask,
        max_length = 196,
        num_beams = 8,
        repetition_penalty = 2.5,
        length_penalty = 2.0,
        early_stopping = True
    )
    
    
    predicted_text = [
        tokenizer.decode(generation_id, skip_special_tokens = True, clean_up_tokenization_spaces = True) for generation_id in generated_ids
    ]
    
    return ''.join(predicted_text)

In [29]:
test_comment = '당사는 본사를 거점으로 한국과 CE, IM 부문 산하 해외 9개 지역총괄 및 DS 부문 산하 해외 5개 지역총괄의 생산ㆍ판매법인, Harman 산하 종속기업 등 241개의 종속기업으로 구성된 글로벌 전자 기업입니다. 사업군별로 보면, Set(세트) 사업군에서는 TV를 비롯하여 모니터, 냉장고, 세탁기, 에어컨 등을 생산ㆍ판매하는 CE(Consumer Electronics) 부문과 HHP(스마트폰 등), 네트워크시스템, 컴퓨터 등을 생산ㆍ판매하는 IM(Information technology & Mobile communications) 부문이 있습니다. 부품 사업군에서는 DRAM, NAND Flash, 모바일AP 등의 제품을 생산ㆍ판매하고 있는 반도체 사업과 모바일ㆍTVㆍ모니터ㆍ노트북 PC용 등의 OLED 및 TFT-LCD 디스플레이 패널을 생산ㆍ판매하고 있는 DP 사업의 DS(Device Solutions) 부문으로 구성되어 있습니다. 또한, 2017년에 인수한 Harman 부문에서 디지털 콕핏(Digital Cockpit), 텔레매틱스(Telematics), 스피커 등을 생산ㆍ판매하고 있습니다. 지역별로 보면, 국내 지역에서는 CE, IM 부문 및 반도체 사업 등을 총괄하는 본사와 29개의 종속기업이 사업을 운영하고 있습니다. 본사는 수원, 구미, 광주, 기흥, 화성, 평택사업장 등으로 구성되어 있으며, 국내 종속기업은 디스플레이 패널을 생산하는 삼성디스플레이㈜와 국내 대리점 판매를 전담하는 삼성전자판매㈜, 제품 서비스를 담당하는 삼성전자서비스㈜ 및 제품 운송을 담당하는 삼성전자로지텍㈜ 등 총 29개의 비상장 종속기업으로 구성되어 있습니다.\
해외 지역(미주, 유럽ㆍCIS, 중동ㆍ아프리카, 아시아 등지)에서는 생산, 판매, 연구 활동 등을 담당하는 212개의 비상장 종속기업이 운영되고 있습니다.\
미주에는 TV, HHP 등 Set제품의 미국 판매를 담당하는 SEA(New Jersey, USA), TV 생산을 담당하는 SII(California, USA), 반도체 생산을 담당하는 SAS(Texas, USA), 전장부품사업 등을 담당하는 Harman(Connecticut, USA) 등을 포함하여 총 55개의 판매ㆍ생산 등을 담당하는 법인이 있습니다.\
유럽ㆍCIS에는 Set제품 판매법인 SEUK(UK), SEG(Germany), SEF(France), SEI(Italy) 등과 TV 생산법인 SEH(Hungary), SESK(Slovakia), 냉장고 등 가전 생산법인 SEPM(Poland) 등을 포함하여 총 75개의 법인이 운영되고 있습니다.\
중동ㆍ아프리카에는 판매ㆍ생산 등을 담당하는 19개 법인이 운영되고 있습니다. \
아시아(중국 제외)에는 SAPL(Singapore)을 중심으로 판매법인 SEAU(Australia), SEPCO(Philippines), SME(Malaysia) 등과, HHP 등 복합생산법인 SIEL(India), HHP 생산법인 SEVㆍSEVT(Vietnam), TV 등 생산법인 SEHC(Vietnam), DP 생산법인 SDV(Vietnam) 등을 포함하여 총 30개의 법인이 운영되고 있습니다.\
중국에는 중국 내 Set제품 판매법인 SCIC(Beijing), SEHK(Hong Kong) 등과 반도체ㆍDP 판매법인 SSS(Shanghai), SSCX(Xian), Set제품 생산법인 SSEC(Suzhou) 등, 반도체 생산법인 SCS(Xian) 등을 포함하여 총 33개의 법인이 운영되고 있습니다.'

In [30]:
summarize(test_comment)

'당사는 본사를 거점으로 한국과 CE, IM 부문 산하 해외 5개 지역총괄의 생산ᆞ판매법인, Harman 산하 종속기업 등 241개의 종속기업으로 구성된 글로벌 전자 기업인 삼성디스플레이(주)와 국내 대리점 판매를 전담하는 삼성전자판매(주), 제품 서비스를 담당하는 삼성전자서비스(주) 및 제품 운송을 담당하는 삼성전자로지텍(주) 등 총 29개의 비상장 종속기업으로 구성되어 있다.'

In [31]:
test_comment = '''집행유예 상태에서 마약을 투여한 혐의로 붙잡힌 가수 연습생 출신 한서희(26)가 또 다시 재판에 넘겨졌다.

9일 더팩트에 따르면 수원지검 성남지청은 한서희를 마약류관리에관한법률위반 혐의로 불구속 기소했으며, 이날 오후 2시 30분 수원지법 성남지원에서 첫 공판이 열린다.

한서희는 지난해 7월 보호관찰소가 불시에 시행한 소변검사에서 메스암페타민(필로폰) 및 암테타민 등 향정신성의약품 양성 반응이 나와 보호관찰소에 구금됐다가 풀려났다. 검찰이 한서희의 집행유예를 취소하는 절차를 밟았으나 한서희가 소변검사 오류를 주장했고 모발검사에서는 음성이 나와 석방된 것으로 알려졌다.

하지만 검찰은 계속 한서희에 대한 조사를 진행했고, 일정 정도 혐의가 소명돼 재판에 넘긴 것으로 전해졌다. 집행유예 기간 중 동종 범죄를 저지른 만큼 실형을 살게 될 가능성도 높아졌다. '''

In [32]:
summarize(test_comment)

'9일 더팩트에 따르면 수원지검 성남지청은 지난해 7월 보호관찰소가 불시에 시행한 소변검사에서 메스암페타민(필로폰) 및 암테타민 등 향정신성의약품 양성 반응이 나와 보호관찰소에 구금됐다가 풀려난 가수 연습생 출신 한서희(26)를 마약류관리에관한법률위반 혐의로 불구속 기소했으며 이날 오후 2시 30분 수원지법 성남지원에서 첫 공판이 열린다.'

In [33]:
test_comment = '''국정농단 사건으로 복역 중인 최서원(개명 전 최순실)씨가 문재인 대통령에게 박근혜 전 대통령의 특별사면을 요청했다.

9일 법조계에 따르면 최씨는 지난달 문 대통령에게 5장짜리 자필편지를 보내 박 전 대통령의 특별사면을 요청했다.

최씨는 편지에서 "박 대통령께서 구금된 지 4년이 넘었다"며 "지친 그분이 제발 자택에서라도 지낼 수 있도록 사면해주길 바란다"고 호소했다.

청와대는 담당 부처인 법무부를 통해 지난달 말 최씨에게 "사면은 대통령 고유 권한 사항이고 이후 사면 업무에 참고하겠다"는 내용의 민원 처리결과 공문을 발송했다.

법무부 관계자는 "사면 관련한 민원이 많아서 통상의 문구대로 회신한 것"이라며 "어떤 방향성을 갖고 검토한 건 아니다"라고 확대 해석을 경계했다.

한편, 박 전 대통령은 지난 2017년 3월 31일 구속됐다. 그는 9일 기준 1532일(만 4년 2개월 9일째)복역 중이다. 박 전 대통령은 지난 1월 14일 대법원 재상고심에서 징역 20년이 확정됐다. 앞서 새누리당 공천 개입 혐의로 징역 2년이 확정된 것까지 박 전 대통령의 최종형량은 징역 22년이다. 오는 2039년 87세 만기 출소 예정이다.'''

In [34]:
summarize(test_comment)

'9일 법조계에 따르면 국정농단 사건으로 복역 중인 최서원(개명 전 최순실)씨가 문재인 대통령에게 5장짜리 자필편지를 보내 박근혜 전 대통령의 특별사면을 요청했으며, 편지에서 "박 대통령께서 구금된 지 4년이 넘었다"며 지친 그분이 제발 자택에서라도 지낼 수 있도록 사면해주길 바란다고 호소했다.'

In [35]:
test_comment = '''
삼성전자가 무선주파수(RF) 첨단 공정 기술을 앞세워 5세대(5G) 이동통신 반도체 파운드리 서비스 확대에 나섰다.

삼성전자는 9일 차세대 '8나노 RF(Radio Frequency) 공정 기술'을 개발했다고 밝혔다.

이를 통해 6㎓ 이하 대역부터 밀리미터파(mmWave)까지 5G 통신 반도체 시장을 적극 공략할 계획이다. 삼성전자는 2015년 28나노 RF 공정 서비스를 시작한 뒤 2017년 업계 최초 양산에 나선 14나노에 이어 이번에 8나노 공정 기술까지 확보했다.

삼성전자 8 나노 RF 파운드리는 멀티 채널·안테나를 지원하는 5G 통신용 RF 칩을 원칩 솔루션으로 제공한다. RF 칩은 모뎀칩에서 나오는 디지털 신호를 아날로그로 변환해 우리가 사용할 수 있는 무선 주파수로 바꿔주고 반대로 모뎀칩으로 전송하기도 하는 무선 주파수 송수신 반도체다.

삼성전자 8나노 RF 공정은 이전 14나노 공정 대비 RF 칩 면적을 약 35% 줄일 수 있다. 전력 효율도 약 35% 향상된다.

또 적은 전력을 사용하면서도 신호를 크게 증폭할 수 있는 RF 전용 반도체 소자 'RFeFET(RF extremeFET)'를 개발, 8나노 RF 공정에 적용했다. RFeFET의 전자가 흐르는 통로인 채널 주변부에 특정 소재를 적용하고 물리적인 자극을 통해 전자 이동 특성을 극대화했다. RF 칩의 전체 트랜지스터 수가 줄어들어 소비전력을 줄일 수 있고, 아날로그 회로 면적도 줄일 수 있다.

이형진 삼성전자 파운드리사업부 기술개발실 마스터는 “공정 미세화와 RF 성능 향상을 동시에 구현한 삼성전자 8나노 기반 RF 파운드리는 소형·저전력·고품질 통신 장점을 갖춰 고객들에 최적 솔루션을 제공할 것"이라며 ”삼성전자는 최첨단 RF 파운드리 경쟁력을 바탕으로 5G를 비롯한 차세대 무선통신 시장을 적극 대응해 나갈 것"이라고 말했다.'''

In [36]:
summarize(test_comment)

"삼성전자는 9일 멀티 채널·안테나를 지원하는 5G 통신용 RF 칩을 원칩 솔루션으로 제공하는 차세대 '8나노 RF(Radio Frequency) 공정 기술'을 개발했다고 밝혔으며, 6GHz 이하 대역부터 밀리미터파까지 5G 통신 반도체 시장을 적극 공략할 계획이다."

In [37]:
test_comment = '''
주주환원정책  강화 LG 유플러스는  자사주  1,000 억원  매입과  중간배당  실시를  공시하였음.  자사주  매입은 창사  이래  최초로  향후  실적  및  주가에  대한  경영진의  자신감으로  해석할  수  있음.  또한 중간배당  실시는  분기별  안정적인  현금흐름을  통해  신규  투자자  유입을  기대할  수  있는 요인임.  LG 유플러스는  경쟁사  대비  주주환원에서  약점이  있었으나  이번  주주환원  강화로 기업가치  상승에  긍정적  영향을  미칠  전망
자사주  매입  및  중간배당  실시  결정 LG 유플러스는 6 월8 일 공시를 통해 자사주 매입을 결정하였다. 매입규모는 1,000 억원으로 현재 LG유플러스의 시가총액 대비 1.5%에 해당하는 금액이다. LG유플러스의 자사주 취득은 창사이래 처음으로 지난달 설립된 ESG 위원회의 사전논의를 거쳐 이사회에서 확정되었다. LG유플러스 주가는 작년 말 대비 31.1% 상승하였음에도 불구하고 자사주 매입을 결정한 것은 주주가치 제고를 위한 의지 표명과 경여진의 실적에 대한 자신감으로 해석할 수 있다. 또한 자사주는 배당에서 제외되기 때문에 일반주주에게 돌아가는 배당재원이 확대되는 효과도 기대할 수 있다. 자사주 매입 결정과 함께 중간배당을 시행하기로 하였다. 중간배당은 올해부터 실시되며, 그 규모에 대해서는 향후 이사회 결의를 통해 결정될 예정이다. 중간배당 결정은 분기별 안정적인 현금흐름에 대한 자신감으로 해석할 수 있다. 또한 배당투자자 및 Long-Term 투자자의 신규 유입을 기대할 수 있다는 측면에서 주가에 긍정적이다.  투자의견  매수,  목표주가  18,000 원(유지) LG 유플러스에 대한 투자의견 매수를 유지한다. 2021 년 영업이익은 1 조원을 넘어설 것으로 기대되는 호실적이 예상되는 가운데 자사주 매입과 중간배당 실시 등 주주환원 정책의 강화로 실적 및 주가에 대한 경영진의 자신감을 확인했기 때문이다. 특히 경쟁사 대비 주주환원 측면에서 약점이 있어 디스카운트 요인이 있던 LG유플러스는 이번 주주환원 강화를 통해 점진적인 기업가치 상승을 기대할 수 있게 되었다. '''

In [38]:
summarize(test_comment)

'LG 유플러스는 경쟁사 대비 주주환원에서 약점이 있었으나 이번 주주환원 강화로 기업가치 상승에 긍정적 영향을 미칠 전망으로 6월8일 공시를 통해 자사주 1,000 억원 매입과 중간배당 실시를 공시하였으며, 그 규모에 대해서는 향후 이사회 결의를 통해 결정될 예정이다.'

In [ ]:
test_comment = '''
(1) 온라인 광고 부문

1) 산업의 특성
온라인 광고는 모바일과 PC, 나아가 IoT를 비롯한 뉴미디어 플랫폼 기반의 디지털 마케팅에 활용되는 광고 상품과 해당 플랫폼을 통해 사용자와 비즈니스, 비즈니스와 비즈니스를 연결하는 행위를 총칭합니다. PC 웹을 중심으로 성장을 시작한 온라인 광고의 태동기에는 사용자들의 검색어에 대응하는 연관 검색결과를 노출 상위에 배치하는 검색광고와 웹서비스 내의 주요 지면을 활용한 디스플레이광고가 주축을 이루었습니다. 이후 디지털 기술 및 인프라 환경의 발전과 디바이스의 보편화에 따른 사용자들의 미디어 소비 행태 변화가 관찰되었고 이러한 흐름에 맞추어 다양한 플랫폼에 적합한 새로운 광고 형태가 지속적으로 등장하며 그 성장세를 이어가고 있습니다. 온라인광고는 전통적인 광고매체 대비하여 상대적으로 정확한 마케팅 성과의 측정이 가능함에 따라 이를 바탕으로 정교한 마케팅 전략의 수립이 가능하고 또한 효율적인 예산 분배를 지원하는 차별점이 있어서 대형 브랜드 광고주와 중소형 사업자들을 아우르면서 다양한 목적의 광고 캠페인를 소화할 수 있습니다.

2) 산업의 성장성
한국방송광고진흥공사에 따르면 2021년 온라인 광고시장은 전년 대비 9.4% 성장한 8조원 규모로 추정됩니다. 특히, 온라인 광고시장은 지난 2017년을 기점으로 PC광고 시장의 규모를 넘어선 모바일 광고 시장의 성장이 지속되면서 전체 온라인광고의 성장을 견인하고 있습니다.

이러한  성장세는 미디어 환경의 변화에 기인하는 것으로 판단하고 있습니다.  현재 스마트폰은 가장 보편적인 디지털 디바이스로 자리잡았으며 모바일 기반의 콘텐츠,서비스 역시 빠른 속도로 확산되며 수많은 이용자들을 모바일 생태계로 이끌고 있습니다. 더불어 세계 최고 수준의 인터넷, 통신 인프라 기반은 시장의 성장을 가속화 시키는 동력이 되어왔습니다. 스마트폰의 보급과 무선통신 인프라의 발달을 통해 이용자들의 미디어 소비의 중심은 TV 등 기존 매체에서 모바일 기기로 급격히 옮겨 갔고, 이에 따라 기존 매체에 집중되었던 광고주 및 마케터들 역시 디지털 미디어에 더욱 많은 관심을 기울이고 있습니다.

동영상, Live 서비스, 메시지, AR/VR 등을 통한 디지털 콘텐츠 소비가 활발해졌을 뿐 아니라 개인 크리에이터 중심의 1인 미디어 역시 활성화 되고 있어 디지털 마케팅 콘텐츠 역시 이에 발맞춰 변화하고 있습니다. 이러한 마케팅 패러다임의 변화로 인해 광고주와 마케터들의 광고 목적에 부합하는 다채로운 마케팅 활동의 전개가 가능해지면서 온라인 광고시장의 참여자가 확대되는 계기가 되고 있습니다.

더불어 온라인광고의 최대 장점으로 꼽히는 효율 ·성과 기반의 광고 관점에서도 인공지능을 활용한 머신러닝 등의 플랫폼 기술 적용을 통해 서비스 이용자들이 일방적으로 광고를 수용하는 것이 아니라 미디어에 대한 직간접적인 참여가 용이해지고 있어 사용자의 기호와 성향을 보다 명확히 반영하는 캠페인의 전개가 가능해졌습니다.
'''

In [ ]:
summarize(test_comment)

In [42]:
test_comment = '''
(1) 온라인 광고 부문

1) 산업의 특성
온라인 광고는 모바일과 PC, 나아가 IoT를 비롯한 뉴미디어 플랫폼 기반의 디지털 마케팅에 활용되는 광고 상품과 해당 플랫폼을 통해 사용자와 비즈니스, 비즈니스와 비즈니스를 연결하는 행위를 총칭합니다. PC 웹을 중심으로 성장을 시작한 온라인 광고의 태동기에는 사용자들의 검색어에 대응하는 연관 검색결과를 노출 상위에 배치하는 검색광고와 웹서비스 내의 주요 지면을 활용한 디스플레이광고가 주축을 이루었습니다. 이후 디지털 기술 및 인프라 환경의 발전과 디바이스의 보편화에 따른 사용자들의 미디어 소비 행태 변화가 관찰되었고 이러한 흐름에 맞추어 다양한 플랫폼에 적합한 새로운 광고 형태가 지속적으로 등장하며 그 성장세를 이어가고 있습니다. 온라인광고는 전통적인 광고매체 대비하여 상대적으로 정확한 마케팅 성과의 측정이 가능함에 따라 이를 바탕으로 정교한 마케팅 전략의 수립이 가능하고 또한 효율적인 예산 분배를 지원하는 차별점이 있어서 대형 브랜드 광고주와 중소형 사업자들을 아우르면서 다양한 목적의 광고 캠페인를 소화할 수 있습니다.

2) 산업의 성장성
한국방송광고진흥공사에 따르면 2021년 온라인 광고시장은 전년 대비 9.4% 성장한 8조원 규모로 추정됩니다. 특히, 온라인 광고시장은 지난 2017년을 기점으로 PC광고 시장의 규모를 넘어선 모바일 광고 시장의 성장이 지속되면서 전체 온라인광고의 성장을 견인하고 있습니다.

이러한  성장세는 미디어 환경의 변화에 기인하는 것으로 판단하고 있습니다.  현재 스마트폰은 가장 보편적인 디지털 디바이스로 자리잡았으며 모바일 기반의 콘텐츠,서비스 역시 빠른 속도로 확산되며 수많은 이용자들을 모바일 생태계로 이끌고 있습니다. 더불어 세계 최고 수준의 인터넷, 통신 인프라 기반은 시장의 성장을 가속화 시키는 동력이 되어왔습니다. 스마트폰의 보급과 무선통신 인프라의 발달을 통해 이용자들의 미디어 소비의 중심은 TV 등 기존 매체에서 모바일 기기로 급격히 옮겨 갔고, 이에 따라 기존 매체에 집중되었던 광고주 및 마케터들 역시 디지털 미디어에 더욱 많은 관심을 기울이고 있습니다.

동영상, Live 서비스, 메시지, AR/VR 등을 통한 디지털 콘텐츠 소비가 활발해졌을 뿐 아니라 개인 크리에이터 중심의 1인 미디어 역시 활성화 되고 있어 디지털 마케팅 콘텐츠 역시 이에 발맞춰 변화하고 있습니다. 이러한 마케팅 패러다임의 변화로 인해 광고주와 마케터들의 광고 목적에 부합하는 다채로운 마케팅 활동의 전개가 가능해지면서 온라인 광고시장의 참여자가 확대되는 계기가 되고 있습니다.

더불어 온라인광고의 최대 장점으로 꼽히는 효율 ·성과 기반의 광고 관점에서도 인공지능을 활용한 머신러닝 등의 플랫폼 기술 적용을 통해 서비스 이용자들이 일방적으로 광고를 수용하는 것이 아니라 미디어에 대한 직간접적인 참여가 용이해지고 있어 사용자의 기호와 성향을 보다 명확히 반영하는 캠페인의 전개가 가능해졌습니다.  또한 마케팅 활동을 통해 형성된 고객과의 연결을 장기적인 관계로 구축 시켜 나갈 수 있는 광고의 지속성에 대한 광고주들의 관심이 증가하고 있어 구매, 예약, 가입을 비롯한 다양한 비즈니스 액션들과 결합을 통해 성장을 거듭할 것으로 기대하고 있습니다.'''

In [ ]:
summarize(test_comment)